# Laboratorium 4
Bartosz Hanc

---

Zadanie polega na implementacji dwóch algorytmów kompresji:

- statycznego algorytmu Huffmana (2 p)
- dynamicznego algorytmu Huffmana (3 p)

Dla każdego z algorytmów należy wykonać następujące zadania:

1. Opracować format pliku przechowującego dane. Zwróć uwagę na dwie kwestie:
    - Liczba bitów wynikowego pliku nie musi być podzielna przez 8, ale z dysku zawsze odczytujemy
      pełne bajty, dlatego ważne jest, aby jakoś rozwiązać ten problem. W przeciwnym razie po
      dekompresji można uzyskać nadmiarowe dane. 
    - Plik wynikowy musi być binarny, tzn. rozwiązanie nie może zakładać, że w pliku tym zapisywane
      są 0 i 1 jako znaki ASCII.

2. Zaimplementować algorytm kompresji i dekompresji danych dla tego formatu pliku.

3. Zmierzyć współczynnik kompresji (wyrażone w procentach: 1 - plik_skompresowany /
   plik_nieskompresowany) dla plików o rozmiarach: 1kB, 10kB, 100kB, 1MB, o różnej zawartości:
    - wybrany przez Ciebie plik tekstowy z projektu Gutenberg,
    - wybrany przez Ciebie plik z kodem źródłowym jądra Linuksa,
    - plik ze znakami losowanymi z rozkładu jednostajnego - należy uwzględnić wszystkie 256
      wartości, a nie tylko znaki drukowalne.

W sumie w punkcie 3 należy przeprowadzić analizę dla łącznie 12 plików (4 rozmiary x 3 typy plików).
Zmierzyć czas kompresji i dekompresji dla plików z punktu 3.

In [9]:
from heapq import heapify, heappop, heappush
from bitarray import bitarray

In [25]:
class Node:
    def __init__(self, char, freq, left=None, right=None):
        self.char = char
        self.freq = freq
        self.left = left
        self.right = right

    def __gt__(self, other):
        return self.freq > other.freq


# Static Huffman algorithm implementation


def build_tree_static(char_freq: dict) -> Node:
    nodes = [Node(char, char_freq[char]) for char in char_freq.keys()]
    heapify(nodes)

    while len(nodes) > 1:
        left = heappop(nodes)
        right = heappop(nodes)
        node = Node(None, left.freq + right.freq, left, right)
        heappush(nodes, node)

    if len(char_freq) == 1:
        char = list(char_freq.keys())[0]
        return Node(None, char_freq[char], Node(char, char_freq[char]), None)

    return nodes[0]


def generate_codes(tree_root: Node) -> dict:
    codes = {}

    def traverse(node, code=""):
        if node.char != None:
            codes[node.char] = code

        if node.left != None:
            traverse(node.left, code + "0")

        if node.right != None:
            traverse(node.right, code + "1")

    traverse(tree_root)

    return codes


def encode_static(text: str) -> tuple[bitarray, Node]:
    char_freq = {char: 0 for char in set(text)}
    for char in text:
        char_freq[char] += 1
        
    tree_root = build_tree_static(char_freq)
    codes = generate_codes(tree_root)
    encoded_text = bitarray()
    for char in text:
        encoded_text.extend(codes[char])

    return encoded_text, tree_root


def decode_static(input: bitarray, tree_root: Node) -> str:
    output = ""
    node = tree_root
    for bit in input:
        node = node.right if bit else node.left
        if node.char != None:
            output += node.char
            node = tree_root

    return output


#### Pomiary współczynnika kompresji i czasów działania

In [21]:
import os
import time


def runtests(infile: str) -> None:
    outfile = infile + ".huff"
    with open(infile, "r", encoding="utf-8") as file:
        text = file.read()

    s = time.time()
    encoded_text, root = encode_static(text)
    e = time.time()
    encode_time = e - s

    s = time.time()
    decoded_text = decode_static(encoded_text, root)
    e = time.time()
    decode_time = e - s

    with open(outfile, "wb") as file:
        encoded_text.tofile(file)

    print(
        f"Compression ratio for file '{infile}':",
        "{:.3f} %\n".format(
            100 * (1 - os.path.getsize(outfile) / os.path.getsize(infile))
        ),
        "Compression time  : {:.3f} s\n".format(encode_time),
        "Decompression time: {:.3f} s\n".format(decode_time),
        f"Encoded == Decoded: {text == decoded_text}\n",
        "___________________",
    )


In [12]:
from random import randint

"""
# Generate files with 256 ASCII characters having uniform distribution

prefix = "test/"

sufix = "-uniform"
for size in (1024, 10 * 1024, 100 * 1024, 1024**2):
    text = ""
    for i in range(size):
        text += chr(randint(0, 255))
    with open(
        prefix + (str(size // 1024) + "kb" if size != 1024**2 else "1mb") + sufix + ".txt", "w",
        encoding="utf-8"
    ) as file:
        file.write(text)
"""

prefix = "test/"
for size in ("1kb", "10kb", "100kb", "1mb"):
    for sufix in ("-gutenberg", "-linux", "-uniform"):
        runtests(prefix + size + sufix + ".txt")


Compression ratio for file 'test/1kb-gutenberg.txt': 46.514 %
 Compression time  : 0.001 s
 Decompression time: 0.000 s
 Encoded == Decoded: True
 ___________________
Compression ratio for file 'test/1kb-linux.txt': 36.626 %
 Compression time  : 0.001 s
 Decompression time: 0.002 s
 Encoded == Decoded: True
 ___________________
Compression ratio for file 'test/1kb-uniform.txt': 35.010 %
 Compression time  : 0.002 s
 Decompression time: 0.000 s
 Encoded == Decoded: True
 ___________________
Compression ratio for file 'test/10kb-gutenberg.txt': 46.643 %
 Compression time  : 0.002 s
 Decompression time: 0.005 s
 Encoded == Decoded: True
 ___________________
Compression ratio for file 'test/10kb-linux.txt': 39.012 %
 Compression time  : 0.003 s
 Decompression time: 0.005 s
 Encoded == Decoded: True
 ___________________
Compression ratio for file 'test/10kb-uniform.txt': 33.701 %
 Compression time  : 0.005 s
 Decompression time: 0.007 s
 Encoded == Decoded: True
 ___________________
Compres

In [31]:
def encode_dynamic(text: str) -> bitarray:
    encoded_text = bitarray()
    char_freq = {}
    
    for i in range(1, 1+len(text)):
        char = text[i-1]
        if char not in char_freq:
            char_freq[char] = 1
            encoded_text.extend(generate_codes(build_tree_static(char_freq))[char])
        else:
            encoded_text.extend(generate_codes(build_tree_static(char_freq))[char])
            char_freq[char] += 1

        print(generate_codes(build_tree_static(char_freq)))

    return encoded_text

encode_dynamic("abracadabra")

        


{'a': '0'}
{'b': '0', 'a': '1'}
{'a': '0', 'r': '10', 'b': '11'}
{'a': '0', 'r': '10', 'b': '11'}
{'r': '00', 'c': '01', 'b': '10', 'a': '11'}
{'a': '0', 'b': '10', 'r': '110', 'c': '111'}
{'a': '0', 'r': '100', 'd': '101', 'c': '110', 'b': '111'}
{'a': '0', 'r': '100', 'd': '101', 'c': '110', 'b': '111'}
{'a': '0', 'r': '100', 'd': '101', 'c': '110', 'b': '111'}
{'r': '00', 'b': '01', 'd': '100', 'c': '101', 'a': '11'}
{'a': '0', 'd': '100', 'c': '101', 'r': '110', 'b': '111'}


bitarray('001000111101011110011')